In [1]:
from simpler import load

data = load('17.txt')

registers, program = data.strip().split('\n\n')
registers = [int(r.split(': ')[1]) for r in registers.split('\n')]
program = tuple(map(int, program.split(': ')[1].split(',')))
program = tuple(zip(program[::2], program[1::2]))

In [2]:
def run(registers, program):
	registers = registers[:]
	instruction_pointer = 0
	while 0 <= instruction_pointer < len(program):
		op, literal = program[instruction_pointer]
		combo = literal if literal <= 3 else registers[literal - 4]
		instruction_pointer += 1
		if op == 0: registers[0] //= 2**combo
		elif op == 1: registers[1] ^= literal
		elif op == 2: registers[1] = combo % 8
		elif op == 3: registers[0] != 0 and (instruction_pointer := literal // 2)
		elif op == 4: registers[1] = registers[1] ^ registers[2]
		elif op == 5: yield combo % 8
		elif op == 6: registers[1] = registers[0] // 2**combo
		elif op == 7: registers[2] = registers[0] // 2**combo

','.join(map(str, run(registers, program)))

'6,5,4,7,1,6,0,3,1'

In [3]:
def run_bitwise(a):  # translation of the submit program to bitwise:
	while True:
		b = a & 0b111  # b is a function of a
		b = b ^ 0b101
		c = a >> b  # c is also a function of a
		b = b ^ 0b110
		a = a >> 3  # each iteration a is shifted right by 3
		b = b ^ c
		yield b & 0b111  # output is b
		if a == 0: break

all(  # sanity check
	tuple(run([n, 0, 0], program)) == tuple(run_bitwise(n))
	for n in range(10000)
)

True

On each iteration, the number output is only dependent on a. That means that we can build it up starting from zero and shifting to find the next 3 bit offset each time.

In [4]:
program_flat = tuple(b for instruction in program for b in instruction)

candidates = [0]
for i in range(len(program_flat)):
	next_candidates = []
	for c in candidates:
		for word in range(1 << 3):
			n = c << 3 | word
			if tuple(run_bitwise(n)) == program_flat[-i - 1:]:
				next_candidates.append(n)
	candidates = next_candidates

min(candidates)

106086382266778